In [1]:
import numpy as np
import pandas as pd
from pypfopt import EfficientFrontier,risk_models,expected_returns,objective_functions
from pypfopt.base_optimizer import BaseConvexOptimizer
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import statsmodels.api as sm
import warnings
warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)


In [2]:
df_info = pd.read_excel('trading-game-data-08112023.xlsx',sheet_name='info')
df_price = pd.read_excel('trading-game-data-08112023.xlsx',sheet_name='price')
df_price.set_index('Date', inplace=True)
df_sp = pd.read_excel('trading-game-data-08112023.xlsx',sheet_name='index-price' )
df_sp.set_index('Date', inplace=True)
df_size = pd.read_excel('trading-game-data-08112023.xlsx',sheet_name='size' )
df_size.set_index('Date', inplace=True)
df_ptb = pd.read_excel('trading-game-data-08112023.xlsx',sheet_name='price-to-book' )
df_ptb.set_index('Date', inplace=True)
df_turnover = df_size = pd.read_excel('trading-game-data-08112023.xlsx',sheet_name='turnover' )
df_turnover.set_index('Date', inplace=True)

In [3]:
log_returns = np.log(df_price / df_price.shift(1))

In [4]:
df_ff = pd.DataFrame()
df_ff['mktrf'] = np.round(np.log(df_sp['S&P 500'] / df_sp['S&P 500'].shift(1))-0.0002,4)
df_ff['SMB']=0
df_ff['HML']=0

In [5]:
for i in range(len(df_size)):
    row_index = df_size.index[i]
    SV = []
    SN = []
    SG = []
    BV = []
    BN = []
    BG = []
    for j in range(len(df_size.columns)):
        if df_size.iloc[i,j] >= df_size.iloc[i].quantile(0.5):
            if df_ptb.iloc[i,j] >= df_ptb.iloc[i].quantile(0.7):
                SG.append(df_size.columns[j])
            elif df_ptb.iloc[i,j] <= df_ptb.iloc[i].quantile(0.3):
                SV.append(df_size.columns[j])
            else:
                SN.append(df_size.columns[j])
        else:
            if df_ptb.iloc[i,j] >= df_ptb.iloc[i].quantile(0.7):
                BG.append(df_size.columns[j])
            elif df_ptb.iloc[i,j] <= df_ptb.iloc[i].quantile(0.3):
                BV.append(df_size.columns[j])
            else:
                BN.append(df_size.columns[j])
                
    SVR = log_returns.loc[row_index,SV].mean()
    SNR = log_returns.loc[row_index,SN].mean()
    SGR = log_returns.loc[row_index,SG].mean()
    BVR = log_returns.loc[row_index,BV].mean()
    BNR = log_returns.loc[row_index,BN].mean()
    BGR = log_returns.loc[row_index,BG].mean()
   
    df_ff['SMB'].iloc[i] = np.round(1/3*(SVR + SNR + SGR)- 1/3 * (BVR + BNR + BGR),4)
    df_ff['HML'].iloc[i] = np.round(1/2*(SVR + BVR)- 1/2 * (BGR + SGR),4)

C:\Users\Jiaqi\AppData\Local\Temp\ipykernel_16040\2030775510.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ff['SMB'].iloc[i] = np.round(1/3*(SVR + SNR + SGR)- 1/3 * (BVR + BNR + BGR),4)
C:\Users\Jiaqi\AppData\Local\Temp\ipykernel_16040\2030775510.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ff['HML'].iloc[i] = np.round(1/2*(SVR + BVR)- 1/2 * (BGR + SGR),4)


In [6]:
df_ffr = pd.concat([df_ff, log_returns], axis=1)

In [7]:
# window_size = 30
# step_size = 15
# rolling = pd.DataFrame(columns=['stock','start','end', 'Beta_mktrf', 'Beta_SMB', 'Beta_HML', 'T-stat_mktrf', 'T-stat_SMB', 'T-stat_HML'])
# for j in range(3,len(df_ffr.columns)):
#     for i in range(1, len(df_ffr) - window_size + 1 - 60, step_size):
#         window = df_ffr.iloc[i:i + window_size]
#         X = window[['mktrf', 'SMB', 'HML']]
#         X = sm.add_constant(X) 
#         y = window.iloc[:, j]
#         model = sm.OLS(y, X).fit()

#         beta_mktrf = model.params['mktrf']
#         beta_SMB = model.params['SMB']
#         beta_HML = model.params['HML']
#         t_stat_mktrf = model.tvalues['mktrf']
#         t_stat_SMB = model.tvalues['SMB']
#         t_stat_HML = model.tvalues['HML']

#         start_date = window.index[0]
#         end_date = window.index[-1]
#         rolling.loc[len(rolling)] = [window.columns[j],start_date,end_date, beta_mktrf, beta_SMB, beta_HML, t_stat_mktrf, t_stat_SMB, t_stat_HML]

In [8]:
ff_coeff = pd.DataFrame(columns=['stock','start','end', 'const','Beta_mktrf', 'Beta_SMB', 'Beta_HML', 'T-stat_mktrf', 'T-stat_SMB', 'T-stat_HML'])
ff_return = pd.DataFrame()
for j in range(3,len(df_ffr.columns)):
    window = df_ffr.iloc[1:-1]
    #out_window = df_ffr.iloc[140:-1]
    X = window[['mktrf', 'SMB', 'HML']]
    X = sm.add_constant(X) 
    y = window.iloc[:, j]
    model = sm.OLS(y, X).fit()
    
  #  X_pred = out_window[['mktrf', 'SMB', 'HML']]
  #  X_pred = sm.add_constant(X_pred)
    beta_mktrf = model.params['mktrf']
    beta_SMB = model.params['SMB']
    beta_HML = model.params['HML']
    t_stat_mktrf = model.tvalues['mktrf']
    t_stat_SMB = model.tvalues['SMB']
    t_stat_HML = model.tvalues['HML']
    intercept = model.params['const']
    
    y_pred = model.predict(X)
    ff_return[df_ffr.columns[j]] = y_pred
    
    start_date = window.index[0]
    end_date = window.index[-1]
    ff_coeff.loc[len(ff_coeff)] = [window.columns[j],start_date,end_date,intercept, beta_mktrf, beta_SMB, beta_HML, t_stat_mktrf, t_stat_SMB, t_stat_HML]

In [9]:
ff_return = ff_return.fillna(-1)
ff_coeff.sort_values('const',ascending = False).head(20)

,stock,start,end,const,Beta_mktrf,Beta_SMB,Beta_HML,T-stat_mktrf,T-stat_SMB,T-stat_HML
339,NVDA,2023-01-03,2023-11-06,0.003382,1.873802,2.245834,-1.327862,9.675010,4.020849,-5.255572
299,META,2023-01-03,2023-11-06,0.003114,1.740302,0.394072,-0.736802,10.589305,0.831440,-3.436630
387,RCL,2023-01-03,2023-11-06,0.002463,1.507576,0.945151,0.200624,9.557211,2.077616,0.974929
311,MPC,2023-01-03,2023-11-06,0.002355,0.541942,1.746441,1.330442,4.055281,4.531420,7.631385
198,GE,2023-01-03,2023-11-06,0.002204,0.935563,-0.046818,0.068563,8.219569,-0.142625,0.461746
334,NRG,2023-01-03,2023-11-06,0.002136,0.974442,-0.544139,0.861013,7.850102,-1.519991,5.317017
166,EQT,2023-01-03,2023-11-06,0.002004,1.121666,0.184198,1.396673,6.587776,0.375123,6.287948
367,PHM,2023-01-03,2023-11-06,0.002003,1.216992,-0.977896,-0.220519,9.244889,-2.575839,-1.284102
18,AIZ,2023-01-03,2023-11-06,0.001912,0.771897,-0.912830,1.073883,6.005716,-2.462677,6.404739
179,FANG,2023-01-03,2023-11-06,0.001904,0.643826,2.177428,1.351563,5.290163,6.203782,8.512868


In [10]:
mu = expected_returns.mean_historical_return(ff_return, log_returns=True,frequency=252,compounding=False,returns_data = True)
S = risk_models.sample_cov(ff_return, log_returns=True,frequency=252,returns_data = True)

rf = 0.05

def optm_func(weights, expected_return, cov_matrix,rf):
    portfolio_risk = np.sqrt(np.dot(weights, np.dot(cov_matrix, weights)))
    portfolio_return = weights @ expected_return
    sharpe_ratio = (portfolio_return - rf) / portfolio_risk
    return -sharpe_ratio

constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1},
               {'type': 'ineq', 'fun': lambda weights: weights})
             #  {'type': 'ineq', 'fun': lambda weights: 0.1-weights})

n_assets = len(mu)
bounds = [(0, 1) for _ in range(n_assets)]
initial_weights = np.ones(n_assets) / n_assets

optimization = minimize(optm_func, initial_weights, args=(mu, S, rf),
                constraints=constraints, bounds=bounds)
optimal_weights = optimization.x
optimal_return = optimal_weights @ mu
optimal_risk = np.sqrt(optimal_weights @ S @ optimal_weights)
optimal_sharpe_ratio = (optimal_return - rf) / optimal_risk
optimal_weights = np.round(optimal_weights,4)

optimal_sharpe = (optimal_return - rf) / optimal_risk

print(optimal_weights)
print("Optimal Portfolio Return:", optimal_return)
print("Optimal Portfolio Risk:", optimal_risk)
print("Optimal Sharpe Ratio:", optimal_sharpe)

C:\Users\Jiaqi\AppData\Roaming\Python\Python311\site-packages\pypfopt\risk_models.py:70: UserWarning: The covariance matrix is non positive semidefinite. Amending eigenvalues.
  warnings.warn(
C:\Users\Jiaqi\AppData\Roaming\Python\Python311\site-packages\pypfopt\risk_models.py:89: UserWarning: Could not fix matrix. Please try a different risk model.
  warnings.warn(


[0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.2758 0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
 0.     0.     0.     0.     0.     0.     0.     0.     0.   

In [11]:
np.savetxt("weights3.txt", optimal_weights)

In [12]:
ef = EfficientFrontier(mu, S,solver="ECOS")
raw_weights = ef.max_sharpe(risk_free_rate=0.05)
cleaned_weights = list(ef.clean_weights().values())
print(cleaned_weights)
ef.portfolio_performance(verbose=True,risk_free_rate=0.05)
ef.save_weights_to_file('weights2.csv')

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.27591, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 